## 학습정리

### 14장. 단순 회귀 분석
* correlation : 두 변수의 선형관계를 갖고 있는지 확인 가능
* 선형관계의 존재 여부를 확인하는 것 외에도 더욱 자세히 살펴볼 필요가 있음

####  14.1 모델
* 회귀 모델 : $y_i = \beta x_i+\alpha + \varepsilon _i$
    * ex. 친구수와 사용자가 사이트에서 보내는 시간의 관계를 설명해주는 모델 
    * $y_i$ : 사용자 i가 매일 사이트에서 보내는 시간
    * $x_i$ : 사용자 i의 친구 수 
    * $\varepsilon$ : 모델이 고려하지 못하는 다른 요소로 발생하는 오류

* 오류 : $\varepsilon_i = y_i - (\beta x_i + \alpha) $

* 최소자승법(least squares) : 에러의 합을 최소화하는 알파와 베타값을 찾음
    * 독립 변수 x의 평균이 주어지면 알파는 종속 변수 y의 평균을 예측
    * 베타는 입력변수가 std(x)만큼 증가한다면 예측값이 correlation(x,y)*std(y)만큼 증가한다는 것을 의미
        * x와 y가 완벽한 양의 상관관계 : x가 1증가할때 y도 1증가
        * 완벽한 음의 상관관계 : x가 증가할때 y는 감소
        * 상관관계가 없을 때 : 베타 =0 아무런 영향이 없음

* 모델이 주어진 데이터에 얼마나 적합한지 알아보기
    * 그래프 시각화
    * 결정계수(R제곱 값) : 종속 변수의 총 변화량 중 모델이 잡아낼 수 있는 변화량의 비율
        * 최소 0, 최대 1(에러0)
        
####  14.2 경사 하강법 사용하기
* 만약 theta = [alpha,beta]로 정하면 경사하강법을 통해 모델을 만들 수 있음

####  14.3 최대가능도 추정법
* 최소 자승법을 사용한 이유
    * 최대가능도 추정법(max-imum likelihood estimation, MLE)
        * 표본데이터 : $v_1,...,v_n$
        * $p(v_1,...,v_n|\theta)$
          * $\theta$를 모른다면 표본이 주어졌을 때 $\theta$가 발생할 가능도(likelihood)로 생각
          * $\to L(\theta|v_1,...,v_n)$
        * $\theta$ : 가능도를 최대화해 주는 값
        * 확률 질량 함수 대신 확률 밀도 함수를 사용하는 연속형 분포에도 이를 동일하게 적용 가능
        * 회귀 분석에서의 오류를 평균 0,표준편차 $\sigma $인 정규분포를 따른다고 가정
            * $ L(\alpha,\beta| x_i,y_i,\sigma ) = \frac{1}{\sqrt{2\pi}\sigma}exp(-(y_i-\alpha-\beta x_i)^2/2\sigma^2)$
            * 전체 데이터에 대한 가능도는 각 데이터의 가능도를 모두 곱한 값
            * 오류의 제곱 값을 최소화하는 알파와 베타가 계산되는 지점이 가능도가 최대화되는 지점
            * 오류 제곱 값 최소화 = 관측 발생 가능도 최대화 

####  14.4 더 공부해 보고 싶다면 
* 15장 다중 회귀 분석

## code

In [1]:
import numpy as np 
from typing import List, Tuple
Vector = List[float]


In [2]:
# 회귀모델
def predict(alpha: float, beta: float, x_i: float) -> float :
    return beta*x_i + alpha

# 오류
def error(alpha: float, beta: float, x_i: float, y_i: float) -> float :
    """실제 결과가 y_i일 때, betaz*x_i+alpha로 계산된 예측값의 오류"""
    return predict(alpha, beta, x_i) - y_i

def sum_of_sqerrors(alpha : float, beta : float, x : Vector, y : Vector) -> float :
    return sum(error(alpha,beta, x_i,y_i)**2 for x_i,y_i in zip(x,y))

#최소 자승법
def least_squares_fit(x : Vector, y : Vector) -> Tuple[float, float] :
    """x와 y가 학습데이터로 주어졌을 때 오류의 제곱 값을 최소화해 주는 알파와 베타를 계산"""
    beta = np.corrcoef(x,y)[0,1] * np.std(y)/np.std(x)
    alpha = np.mean(y) - beta*np.mean(x)
    return alpha, beta


In [24]:
# numpy에서 두 변수의 상관계수
np.corrcoef(x,y)[0,1]

1.0

In [5]:
x = [i for i in range(-100,110,10)]
y = [3*i-5 for i in x]

# y=3x-5를 찾아내기 
least_squares_fit(x,y)

# y = -5 + 3x으로 예측가능 

(-5.0, 3.0)

In [3]:
def total_sum_of_squares(y : Vector) -> float :
    """평균을 기준으로 y_i의 변화량을 제곱한 값의 총합"""
    return sum( v**2 for v in x-np.mean(x))

def r_squared(alpha: float, beta: float, x: Vector, y: Vector) -> float :
    """모델이 잡아낼 수 있는 y의 변화량의 비율은 1 - 모델이 잡아내지 못하는 y의 변화량의 비율로 계산"""
    return 1.0 - (sum_of_sqerrors(alpha, beta, x, y) / total_sum_of_squares(y))


In [7]:
alpha = -3
beta = 3.5
rsq = r_squared(alpha, beta, x, y)
rsq

0.748909090909091

In [12]:
# 오차가 없을 때 최댓값 = 1
alpha = -5
beta = 3
rsq = r_squared(alpha, beta, x, y)
rsq

1.0

In [16]:
# 경사 하강법으로 모델 만들기
import random
import tqdm

num_epochs = 10000
random.seed(0)

guess = [random.random(), random.random()] # 임의의 위치에서 출발
learning_rate = 0.00001

with tqdm.trange(num_epochs) as t :
    for _ in t :
        alpha,beta = guess
        # 알파에 대한 손실함수의 편미분 : error^2의 미분 2*error
        grad_a = sum(2*error(alpha, beta, x_i,y_i)
                    for x_i, y_i in zip(x, y))
        
        grad_b = sum(2*error(alpha, beta, x_i,y_i)*x_i
                    for x_i,y_i in zip(x, y))
        
        # tqdm 설명에 넣기 위해 손실 계산
        loss = sum_of_sqerrors(alpha,beta, x, y)
        t.set_description(f"loss: {loss:.3f}")
        
        # 최종적으로, 추측 갱신
        guess = gradient_step(guess, [grad_a, grad_b], -learning_rate)
        
alpha, beta = guess

loss: 387778.474:   0%|          | 0/10000 [00:00<?, ?it/s]


NameError: name 'gradient_step' is not defined